## Project : Image Based-Emotion Facial Recognition
##### Dataset : https://www.kaggle.com/datasets/msambare/fer2013

In [ ]:
#Importing all necessary Libraries
import os
import glob
import random
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, InceptionV3, ResNet50, VGG16

from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input

from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler


In [ ]:
# Define the directory location for the dataset
location = "old/data"

# Get the list of datasets in the specified location
name_of_dataset = os.listdir(location)

# Create a list of emotions based on the contents of the 'train' directory
emotion_list = os.listdir(location + "/train")

# Print the list of dataset names and emotions
print(name_of_dataset)
print(emotion_list)

### Visualization of Emotion Datasets

In [ ]:
# Initialize empty lists to store image data and labels
images = []
target_lbl = []
target_idx = []

# Set a random seed for reproducibility
random.seed(42)

# Define the number of rows and columns for the subplots
num_rows = 4
num_cols = 7

# Create a grid of subplots with a larger figure size
fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 7))

# Define the list of emotions
emotions = emotion_list

# Initialize a dictionary to count the occurrences of each emotion
emotion_counts = {}

# Iterate through the emotions
for i, emotion in enumerate(emotions):
    # Get a list of image file paths for the current emotion
    image_files = glob.glob(location + f'/train/{emotion}/*')
    all_image_files = []
    
    # Count and store the number of images for each emotion
    emotion_counts[emotion] = len(image_files)
    
    # Shuffle the image file list
    random.shuffle(image_files)
    
    # Display up to 4 randomly selected images for each emotion
    for j in range(min(4, len(image_files))):
        file_name = image_files[j]
        img = cv2.imread(file_name)
        row = j
        col = i
        axs[row, col].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[row, col].set_xticks([])
        axs[row, col].set_yticks([])
        axs[row, col].set_xlabel(emotion)

# Add a title
plt.suptitle("Visualization of Emotion Datasets", fontsize=12)

plt.tight_layout()
plt.show()


### Training Class Distribution in the Dataset.

In [ ]:
# Sort the emotion counts by values (counts)
sorted_emotion_counts = {k: v for k, v in sorted(emotion_counts.items(), key=lambda item: item[1])}

# Define a list of colors for each emotion
colors = ['#B2C8BA', '#80B3FF', '#D988B9', '#BC7AF9', '#9ED2BE', '#7895CB', 'cyan', 'magenta']

# Create a horizontal bar chart for sorted emotion counts with custom colors
plt.figure(figsize=(10, 6))
bars = plt.barh(y=list(sorted_emotion_counts.keys()), width=list(sorted_emotion_counts.values()), color=colors)
plt.xlabel('Count')
plt.ylabel('Emotion')
plt.title('Value Counts per Emotion')
plt.gca().invert_yaxis() 

# Add text labels for each bar
for bar in bars:
    plt.text(bar.get_width() + 180, bar.get_y() + bar.get_height() / 2, str(int(bar.get_width())), ha='center', va='center')

plt.show()

In [ ]:
emotion_counts = {
    'Angry': 2369,
    'Disgust': 348,
    'Fear': 4097,
    'Happy': 7215,
    'Sad': 4830,
    'Surprise': 3170,
    'Neutral': 4965
}

# labels and counts for the pie chart
labels = emotion_counts.keys()
counts = emotion_counts.values()

# Custom colors
colors = ['#B2C8BA', '#80B3FF', '#D988B9', '#BC7AF9', '#9ED2BE', '#7895CB', 'cyan', 'magenta']

plt.figure(figsize=(6, 6))
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Class Distribution')

plt.show()

### Image Shape Checking

In [ ]:
# Path to train image files
path = "data/train/"

# Initialize an empty list to store image arrays
image_arrays = []

# Use glob to find all image files in subdirectories
image_files = glob.glob(os.path.join(path, '**/*.jpeg'), recursive=True)

# Read and store each image in the list
for image_file in image_files:
    img = cv2.imread(image_file)
    image_arrays.append(img)

    # Normalize the pixel values to a range between 0 and 1
    normalized_img = img / 255.0

# Initialize a flag to check if all images have the same shape
same_shape = True

# Iterate through the image arrays
for i in range(1, len(image_arrays)):
    if np.shape(image_arrays[i]) != np.shape(image_arrays[0]):
        same_shape = False
        break

# Check if all images have the same shape
if same_shape:
    print("All images have the same shape.")
else:
    print("Not all images have the same shape.")


### Emotion Dataset Information and Statistic

In [ ]:
# Create a list to store dataset information
dataset_info = []

for emotion, count in emotion_counts.items():
    dataset_info.append({'Emotion': emotion, 'Count': count})

# Create a DataFrame from the dataset information
dataset_df = pd.DataFrame(dataset_info)

# Descriptive statistics for your dataset
statistics = dataset_df.describe()
print(statistics)

The dataset contains information about seven different emotions. On average, each emotion is represented by approximately 3,856 images, with some having as few as 348 and others as many as 7,215 images.

### Data Preprocessing and Augmentation for Training and Testing

In [ ]:
# Define the paths to your training and testing data directories
train_dir = 'data/train/'
test_dir = 'data/test/'

# Create an image data generator for training data with various data augmentation
training_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to a range of 0 to 1
    validation_split=0.2,  # Split data for validation (20%)
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    rotation_range=5,  # Randomly rotate images by up to 5 degrees
    shear_range=0.2,  
    horizontal_flip=True,  
    vertical_flip=True,  
    fill_mode='nearest'  # Fill missing pixels with the nearest value
)

# Create an image data generator for validation data and testing data
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
testing_datagen = ImageDataGenerator(rescale=1./255)

# Define the image size and batch size
image_size = 48
batch_size = 128

# Create training, validation, and testing datasets using the data generators
training_dataset = training_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(image_size, image_size),
    class_mode='categorical',
    color_mode='grayscale',  
    subset='training',
    batch_size=batch_size
)

validation_dataset = validation_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(image_size, image_size),
    class_mode='categorical',
    color_mode='grayscale', 
    subset='validation',
    batch_size=batch_size
)

testing_dataset = testing_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(image_size, image_size),
    class_mode='categorical',
    color_mode='grayscale',  
    batch_size=batch_size
)

### Model 1: Custom CNN Model

In [ ]:
# Number of classes and epochs
num_classes = 7 
epochs = 50

# Initialize the CNN model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(32, (3, 3), input_shape=(48, 48, 1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

# 2nd Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

# 3rd Convolutional Layer
model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

# 4th Convolutional Layer with L2 regularization
model.add(Conv2D(512, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

# 5th Convolutional Layer with L2 regularization
model.add(Conv2D(512, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.20))


# Flatten the feature maps
model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Fully Connected Layer 2
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Fully Connected Layer 3 (Output Layer)
model.add(Dense(num_classes, activation='softmax'))


# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=10,  
    restore_best_weights=True 
)


# learning rate schedule
def lr_schedule(epoch):
    if epoch < 20:
        return 0.01
    elif epoch < 35:
        return 0.001
    else:
        return 0.0001

lr_scheduler = LearningRateScheduler(lr_schedule)


# Compile the custom_CNN_model
model.compile(
    loss='categorical_crossentropy',  
    optimizer=Adam(learning_rate=0.001),  
    metrics=['accuracy'] 
)


model.summary()

In [ ]:
# Train the model
history = model.fit(
    x=training_dataset, 
    epochs=epochs,  
    validation_data=validation_dataset, 
    callbacks=[early_stopping, lr_scheduler] 
)

### Model 1: Evaluation and Training History for the Custom CNN Model

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(testing_dataset)
print(f"Test Accuracy: {test_accuracy}")

# Get the training and validation loss and accuracy from the training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Plot the training and validation loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend() 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.legend() 
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

### Model 1: Evaluation and Confusion Matrix for Custom CNN Model

In [ ]:
# Make predictions on the testing dataset
test_pred = model.predict(testing_dataset)
test_pred_classes = np.argmax(test_pred, axis=1)
true_classes = testing_dataset.classes

# Generate and print the classification report
class_labels = list(testing_dataset.class_indices.keys())
report = classification_report(true_classes, test_pred_classes, target_names=class_labels, zero_division=1)
print(report)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, test_pred_classes)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Custom CNN Confusion Matrix')
plt.show()



# Make predictions on the testing dataset
predictions = model.predict(testing_dataset)

# Convert one-hot encoded labels to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels from the testing dataset
true_classes = testing_dataset.classes

# Calculate precision, recall, and F1 score for the entire model
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print the precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

### Model 2: MobileNet

In [ ]:
# Image Data Generators Configuration for MobileNet Model 
image_size = 75
batch_size = 128

training_dataset = training_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(image_size, image_size),
    class_mode='categorical',
    subset='training',
    batch_size=batch_size
)

validation_dataset = validation_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(image_size, image_size),
    class_mode='categorical',
    subset='validation',
    batch_size=batch_size
)

testing_dataset = testing_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(image_size, image_size),
    class_mode='categorical',
    batch_size=batch_size
)

# Number of classes for your classification task
num_classes = 7 

# Load the pre-trained MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model with additional dense layers and dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the custom top model
mobile_net_model = Model(inputs=base_model.input, outputs=predictions)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model
mobile_net_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
mobile_net_model.summary()

In [ ]:
# Define the number of training steps and validation steps
train_steps = training_dataset.samples // batch_size
val_steps = validation_dataset.samples // batch_size

# Train the model 
history = mobile_net_model.fit(
    training_dataset,
    steps_per_epoch=train_steps,
    validation_data=validation_dataset,
    validation_steps=val_steps,
    epochs=epochs,  
    callbacks=[early_stopping]  # Include early stopping callback
)

### Model 2: Evaluation and Training History for MobileNet Model

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = mobile_net_model.evaluate(testing_dataset)
print(f"Test Accuracy: {test_accuracy}")

# Get the training and validation loss and accuracy from the training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Plot the training and validation loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend() 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.legend() 
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

### Model 2: Evaluation and Confusion Matrix for MobileNet Model

In [ ]:
# Make predictions on the testing dataset
test_pred = mobile_net_model.predict(testing_dataset)
test_pred_classes = np.argmax(test_pred, axis=1)
true_classes = testing_dataset.classes

# Generate and print the classification report
class_labels = list(testing_dataset.class_indices.keys())
report = classification_report(true_classes, test_pred_classes, target_names=class_labels, zero_division=1)
print(report)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, test_pred_classes)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()




# Make predictions on the testing dataset
predictions = mobile_net_model.predict(testing_dataset)

# Convert one-hot encoded labels to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels from the testing dataset
true_classes = testing_dataset.classes

# Calculate precision, recall, and F1 score for the entire model
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print the precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

### Model 3: Inception3

In [ ]:
# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model with additional dense layers and dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the custom top model
model = Model(inputs=base_model.input, outputs=predictions)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

In [ ]:
# Define the number of training steps and validation steps
train_steps = training_dataset.samples // batch_size
val_steps = validation_dataset.samples // batch_size

# Train the model
history = model.fit(
    training_dataset,
    steps_per_epoch=train_steps,
    validation_data=validation_dataset,
    validation_steps=val_steps,
    epochs=epochs, 
    callbacks=[early_stopping]  # Include early stopping callback
)

### Model 3: Evaluation and Training History for Inception3 Model

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(testing_dataset)
print(f"Test Accuracy: {test_accuracy}")

# Get the training and validation loss and accuracy from the training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Plot the training and validation loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend() 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.legend() 
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

### Model 3: Evaluation and Confusion Matrix for Inception3 Model

In [ ]:
# Make predictions on the testing dataset
test_pred = model.predict(testing_dataset)
test_pred_classes = np.argmax(test_pred, axis=1)
true_classes = testing_dataset.classes

# Generate and print the classification report
class_labels = list(testing_dataset.class_indices.keys())
report = classification_report(true_classes, test_pred_classes, target_names=class_labels, zero_division=1)
print(report)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, test_pred_classes)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()





# Make predictions on the testing dataset
predictions = model.predict(testing_dataset)

# Convert one-hot encoded labels to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels from the testing dataset
true_classes = testing_dataset.classes

# Calculate precision, recall, and F1 score for the entire model
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print the precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

### Model 4: ResNet-50

In [ ]:
image_size = 48
batch_size = 128
training_dataset  = training_datagen.flow_from_directory(directory = train_dir,
                                                   target_size = (image_size, image_size),
                                                   class_mode = 'categorical',
                                                   #color_mode = "grayscale",
                                                   subset = 'training',
                                                   batch_size = batch_size)

validation_dataset = validation_datagen.flow_from_directory(directory = train_dir,
                                                  target_size = (image_size, image_size),
                                                  class_mode = 'categorical',
                                                  #color_mode = "grayscale",
                                                  subset = 'validation',
                                                  batch_size = batch_size)

testing_dataset = testing_datagen.flow_from_directory(directory = test_dir,
                                                target_size = (image_size, image_size),
                                                class_mode = 'categorical',
                                               # color_mode = "grayscale",
                                                batch_size = batch_size)






num_classes = 7  

# Load the pre-trained ResNet-50 model 
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model with additional dense layers and dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  
x = Dense(512, activation='relu')(x)  
x = Dropout(0.5)(x)  
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the custom top model
model = Model(inputs=base_model.input, outputs=predictions)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

In [ ]:
# Define the number of training steps and validation steps
train_steps = training_dataset.samples // batch_size
val_steps = validation_dataset.samples // batch_size

# Train the model
history = model.fit(
    training_dataset,
    steps_per_epoch=train_steps,
    validation_data=validation_dataset,
    validation_steps=val_steps,
    epochs=epochs, 
    callbacks=[early_stopping]  
)

# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(testing_dataset)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
### Model 4: Evaluation and Training History for ResNet-50 Model

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(testing_dataset)
print(f"Test Accuracy: {test_accuracy}")

# Get the training and validation loss and accuracy from the training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Plot the training and validation loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend() 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.legend() 
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

### Model 4: Evaluation and Confusion Matrix for ResNet-50 Model

In [ ]:
# Make predictions on the testing dataset
test_pred = model.predict(testing_dataset)
test_pred_classes = np.argmax(test_pred, axis=1)
true_classes = testing_dataset.classes

# Generate and print the classification report
class_labels = list(testing_dataset.class_indices.keys())
report = classification_report(true_classes, test_pred_classes, target_names=class_labels, zero_division=1)
print(report)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, test_pred_classes)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()




# Make predictions on the testing dataset
predictions = model.predict(testing_dataset)

# Convert one-hot encoded labels to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels from the testing dataset
true_classes = testing_dataset.classes

# Calculate precision, recall, and F1 score for the entire model
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print the precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

### Model 5: VGG-16

In [ ]:
num_classes = 7 

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model with additional dense layers and dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the custom top model
model = Model(inputs=base_model.input, outputs=predictions)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

In [ ]:
# Define the number of training steps and validation steps
train_steps = training_dataset.samples // batch_size
val_steps = validation_dataset.samples // batch_size

# Train the model 
history = model.fit(
    training_dataset,
    steps_per_epoch=train_steps,
    validation_data=validation_dataset,
    validation_steps=val_steps,
    epochs=epochs,  
    callbacks=[early_stopping] 
)

### Model 5: Evaluation and Training History for VGG-16 Model

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(testing_dataset)
print(f"Test Accuracy: {test_accuracy}")

# Get the training and validation loss and accuracy from the training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Plot the training and validation loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend()  
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.legend()  
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()


### Model 5: Evaluation and Confusion Matrix for VGG-16 Model

In [ ]:
# Make predictions on the testing dataset
test_pred = model.predict(testing_dataset)
test_pred_classes = np.argmax(test_pred, axis=1)
true_classes = testing_dataset.classes

# Generate and print the classification report
class_labels = list(testing_dataset.class_indices.keys())
report = classification_report(true_classes, test_pred_classes, target_names=class_labels, zero_division=1)
print(report)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, test_pred_classes)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()




# Make predictions on the testing dataset
predictions = model.predict(testing_dataset)

# Convert one-hot encoded labels to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels from the testing dataset
true_classes = testing_dataset.classes

# Calculate precision, recall, and F1 score for the entire model
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print the precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

### Performance Comparison

In [ ]:
import matplotlib.pyplot as plt

# Model names and corresponding test accuracy values
model_names = ["Custom CNN Model", "ResNet50", "MobileNet", "VGG16", "Inception3"]
test_accuracy = [29.17, 24.71, 42.98, 38.62, 34.01]

# Original colors for each model
original_colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightsalmon', 'lightpink']

# Sort the models by test accuracy in descending order
sorted_data = sorted(zip(model_names, test_accuracy, original_colors), key=lambda x: x[1], reverse=True)
model_names_sorted, test_accuracy_sorted, original_colors_sorted = zip(*sorted_data)

# Create a bar chart with original colors
plt.figure(figsize=(10, 6))
bars = plt.bar(model_names_sorted, test_accuracy_sorted, color=original_colors_sorted)
plt.xlabel('Model')
plt.ylabel('Test Accuracy (%)')
plt.title('Test Accuracy of Different Models')
plt.xticks(rotation=45, ha='right')

# Display the test accuracy values on top of the bars
for i, acc in enumerate(test_accuracy_sorted):
    plt.text(model_names_sorted[i], acc + 1, f'{acc:.2f}%', ha='center')

plt.tight_layout()
plt.show()

### Save the trained model to a file

In [ ]:
mobile_net_model.save('my_best_model.h5')